In [18]:
import plotly.offline as py

In [19]:
py.init_notebook_mode()

In [20]:
import plotly.graph_objs as go

In [21]:
import numpy as np

In [13]:
x = np.random.randn(2000)
y = np.random.randn(2000)
py.iplot([go.Histogram2dContour(x=x, y=y, contours=Contours(coloring='heatmap')),
       Scatter(x=x, y=y, mode='markers', marker=Marker(color='white', size=3, opacity=0.3))], show_link=False)

In [1]:
from netCDF4 import Dataset

In [7]:
PATH='/mnt/web/brey/2016H/9/13/00/'

In [8]:
tag='med'

In [9]:
d = Dataset(PATH+'trim-'+tag+'.nc')

h = d.variables['S1'][:]
try:
    u = d.variables['WINDU'][:]
    v = d.variables['WINDV'][:]
except:
    print 'no wind'
    pass
uw = d.variables['U1'][:]
vw = d.variables['V1'][:]
lon = d.variables['XCOR'][:]
lat = d.variables['YCOR'][:]


In [11]:
from plotly.tools import FigureFactory as FF

In [27]:
vectors = FF.create_quiver(lon[::10,::10], lat[::10,::10], u[0,::10,::10], v[0,::10,::10],
                       scale=.25,
                       arrow_scale=.4,
                       name='quiver',
                       line=dict(width=1))


In [28]:
layout = dict(
        title = 'med',
        showlegend = False,
        width=650,
        height=600,
        geo = dict(
            showland = True,
            showlakes = True,
            showcountries = True,
            showocean = True,
            countrywidth = 0.5,
            landcolor = 'rgb(230, 145, 56)',
            lakecolor = 'rgb(0, 255, 255)',
            oceancolor = 'rgb(0, 255, 255)',
            projection = dict(
                type = 'orthographic',
                rotation = dict(
                    lon = -100,
                    lat = 40,
                    roll = 0
                )
            ),
            lonaxis = dict(
                showgrid = True,
                gridcolor = 'rgb(102, 102, 102)',
                gridwidth = 0.5
            ),
            lataxis = dict(
                showgrid = True,
                gridcolor = 'rgb(102, 102, 102)',
                gridwidth = 0.5
            )
        )
    )


In [29]:
py.iplot(vectors,layout)

In [53]:
contours = [{
        'type': u'contour',
        'x': lon[::10,::10][:,0],
        'y': lat[::10,::10][0,:],
        'z': h[-1,::10,::10],
       }
        ]

In [64]:
layout = dict(
        title = 'med',
        showlegend = False,
        width=650,
        height=600,
        geo = dict(
            showland = True,
            showlakes = True,
            showcountries = True,
            showocean = True,
            countrywidth = 0.5,
            landcolor = 'rgb(230, 145, 56)',
            lakecolor = 'rgb(0, 255, 255)',
            oceancolor = 'rgb(0, 255, 255)',
            projection = dict(
                type = 'natural earth',
                rotation = dict(
                    lon = -100,
                    lat = 40,
                    roll = 0
                )
            ),
            lonaxis = dict(
                showgrid = True,
                gridcolor = 'rgb(102, 102, 102)',
                gridwidth = 0.5
            ),
            lataxis = dict(
                showgrid = True,
                gridcolor = 'rgb(102, 102, 102)',
                gridwidth = 0.5
            )
        ),
             colorbar = dict(
             nticks =  10,
             ticks = 'outside',
             ticklen = 5,
             tickwidth = 1,
             showticklabels = True,
             tickangle = 0,
             tickfont = dict(
                size = 12
            )
         )
    )


In [66]:
py.iplot(contours, layout)

In [82]:
trace1 = go.Contour(
        x = lon[::10,::10][:,0],
        y = lat[::10,::10][0,:],
        z = h[-1,::10,::10],
        colorscale="RdBu",
        zauto=False,  # custom contour levels
        zmin=-5,      # first contour level
        zmax=5        # last contour level  => colorscale is centered about 0
)

In [67]:
from mpl_toolkits.basemap import Basemap

In [68]:
m = Basemap() 

In [76]:
# Make trace-generating function (return a Scatter object)
def make_scatter(x,y):
    return go.Scatter(
        x=x,
        y=y,
        mode='lines',
        line=go.Line(color="black"),
        name=' '  # no name on hover
    )

In [70]:
# Functions converting coastline/country polygons to lon/lat traces
def polygons_to_traces(poly_paths, N_poly):
    ''' 
    pos arg 1. (poly_paths): paths to polygons
    pos arg 2. (N_poly): number of polygon to convert
    '''
    traces = []  # init. plotting list 

    for i_poly in range(N_poly):
        poly_path = poly_paths[i_poly]
        
        # get the Basemap coordinates of each segment
        coords_cc = np.array(
            [(vertex[0],vertex[1]) 
             for (vertex,code) in poly_path.iter_segments(simplify=False)]
        )
        
        # convert coordinates to lon/lat by 'inverting' the Basemap projection
        lon_cc, lat_cc = m(coords_cc[:,0],coords_cc[:,1], inverse=True)
        
        # add plot.ly plotting options
        traces.append(make_scatter(lon_cc,lat_cc))
     
    return traces

In [71]:
# Function generating coastline lon/lat traces
def get_coastline_traces():
    poly_paths = m.drawcoastlines().get_paths() # coastline polygon paths
    N_poly = 91  # use only the 91st biggest coastlines (i.e. no rivers)
    return polygons_to_traces(poly_paths, N_poly)

In [72]:
# Function generating country lon/lat traces
def get_country_traces():
    poly_paths = m.drawcountries().get_paths() # country polygon paths
    N_poly = len(poly_paths)  # use all countries
    return polygons_to_traces(poly_paths, N_poly)

In [77]:
# Get list of of coastline and country lon/lat traces
traces_cc = get_coastline_traces()+get_country_traces()

In [83]:
data = go.Data([trace1]+traces_cc)

In [87]:
title = u"Average daily surface air temperature anomalies [\u2103]<br> \
in July 2014 with respect to 1981-2010 climatology"

anno_text = "Data courtesy of \
<a href='http://www.esrl.noaa.gov/psd/data/composites/day/'>\
NOAA Earth System Research Laboratory</a>"

axis_style = dict(
    zeroline=False,
    showline=False,
    showgrid=False,
    ticks='',
    showticklabels=False,
)

layout = go.Layout(
    title=title,
    showlegend=False,
    hovermode="closest",        # highlight closest point on hover
    xaxis=go.XAxis(
        axis_style,
        range=[lon[0],lon[-1]]  # restrict y-axis to range of lon
    ),
    yaxis=go.YAxis(
        axis_style,
    ),
    annotations=go.Annotations([
        go.Annotation(
            text=anno_text,
            xref='paper',
            yref='paper',
            x=0,
            y=1,
            yanchor='bottom',
            showarrow=False
        )
    ]),
    autosize=False,
    width=1000,
    height=500,
)

In [89]:
fig = go.Figure(data=data, layout=layout)



In [92]:
py.iplot(fig)

In [110]:
data2=([vectors.data[0]]+traces_cc)

In [111]:
fig2 = go.Figure(data=data2, layout=layout)



In [112]:
py.iplot(fig2)